In [20]:
import pandas as pd

In [21]:
df = pd.read_parquet(path='../data/train.parquet', engine='pyarrow')

In [22]:
df.head()

,programevent_id,channel,market,turnover,weekday,period,date,time,participant_a,participant_b
0,33954710,Retail,standard,17795.0,Friday,Evening,2009-09-18,20:30:00,Schalke 04,VfL Wolfsburg
1,33954710,Retail,double-chance,4950.0,Friday,Evening,2009-09-18,20:30:00,Schalke 04,VfL Wolfsburg
2,33954710,Retail,next-point,4211.0,Friday,Evening,2009-09-18,20:30:00,Schalke 04,VfL Wolfsburg
3,33954710,Retail,points-more-less-than,2031.0,Friday,Evening,2009-09-18,20:30:00,Schalke 04,VfL Wolfsburg
4,33954710,Retail,points-more-less-rest,2587.0,Friday,Evening,2009-09-18,20:30:00,Schalke 04,VfL Wolfsburg


In [23]:
features = ['channel', 'market', 'weekday', 'period', 'participant_a', 'participant_b']

In [24]:
df = pd.get_dummies(data=df, prefix='is_', columns=features).drop(labels=['programevent_id', 'date', 'time'], axis=1)

In [25]:
import numpy as np

In [26]:
features = np.delete(arr=df.columns.values, obj=np.where(df.columns.values == 'turnover'))

In [27]:
df.head()

,turnover,is__Online,is__Retail,is__corner-kick-handicap,is__corner-kicks-more-less-rest,is__corner-kicks-more-less-rest-ht,is__corner-kicks-more-less-than,is__correct-score-live,is__custom-place-yes-no,is__custom-point-yes-no,...,is__Hertha BSC,is__Hoffenheim,is__Kaiserslautern,is__RB Leipzig,is__SC Freiburg,is__SC Paderborn,is__Schalke 04,is__VfB Stuttgart,is__VfL Wolfsburg,is__Werder Bremen
0,17795.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,4950.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,4211.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2031.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,2587.0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [33]:
import random as rand
from sklearn.sklearn import DecisionTreeRegressor

In [29]:
# cross-validation testing
split = rand.uniform(0.60, 0.80)

train_size = int(len(df) * split)

train, test = df[0:train_size], df[train_size:len(df)]

In [30]:
# values of features
X = np.array(train[features].values)

# target values
y = list(train['turnover'])

In [34]:
mdl = DecisionTreeRegressor().fit(X, y)
print(mdl)

# make predictions
pred = mdl.predict(test[features])

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)


In [35]:
from sklearn.metrics import r2_score

# summarize the fit of the model
r2_score(test['turnover'].values, pred)

-0.11173413055624093

In [36]:
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(test['turnover'].values, pred)
rms

610972071.993002

In [30]:
results = pd.DataFrame(data=dict(original=test['turnover'], prediction=pred), index=test.index)

In [31]:
results

,original,prediction
31286,73290.0,71992.000000
31287,11006.0,6536.000000
31288,2856.0,1779.000000
31289,5372.0,5852.333333
31290,2329.0,1764.333333
31291,9137.0,11044.666667
31292,10249.0,6326.000000
31293,1104.0,1730.666667
31294,2633.0,2106.333333
31295,6958.0,12891.000000
